In [1]:
import asyncio
import websockets
import json
import time

import sys
import pandas as pd
import requests
from web3 import Web3

import libs.common.utils as utils
import libs.common.payload as payload
import config.config as cfg

In [2]:
func_calls = []

In [4]:
async def process_message(block_info):
    val = payload.eth_logs_by_block_payload(0, block_info["number"], block_info["number"])
    resp = requests.post(cfg.config["http_url"], json=val)
    
    df_logs = pd.DataFrame(resp.json()["result"])
    df_logs["timestamp"] = block_info["timestamp"]
    log_sig = "Transfer(index_topic_1 address from, index_topic_2 address to, uint256 value)"
    log_hash, log_name, log_topics, log_data = utils.parse_log_signature(log_sig)
    df_logs = utils.parse_log_data(df_logs.loc[df_logs["topics"].str[0] == log_hash].copy(), log_topics, log_data, log_name = log_name)
    
    print(len(resp.json()["result"]), df_logs.shape[0])

In [9]:
async def message():
    async with websockets.connect(cfg.config["ws_url"]) as ws:
        sub_newheads = { "id":1, "jsonrpc":"2.0", "method":"eth_subscribe", "params":["newHeads"] }
        await ws.send(json.dumps(sub_newheads))
        sub_res = await ws.recv()
        print(json.loads(sub_res))
        while True:
            message = await asyncio.wait_for(ws.recv(), timeout=60)
            block_info = json.loads(message)["params"]["result"]
            start = time.time()
            await process_message(block_info)
            end = time.time()
            print(end-start)

In [ ]:
import nest_asyncio
nest_asyncio.apply()

loop = asyncio.get_event_loop()
while True:
    loop.run_until_complete(message())

{'jsonrpc': '2.0', 'id': 1, 'result': '0x2e360b65a8103f57f7be85a03d909c27'}
287 131
1.5188450813293457
354 138
1.3842298984527588
